In [93]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import random
from colorama import Fore, Style, init
init(autoreset=True)

### 1. Degree Anonymization (Dynamic Programming & Greedy)

In [94]:
import numpy as np

class DegreeAnonymizer:
    def __init__(self, degrees, k):
        # Store the original degrees and their sorted order with indices
        self.original_degrees = degrees.copy()
        self.sorted_nodes = sorted(range(len(degrees)), key=lambda x: degrees[x], reverse=True)
        self.sorted_degrees = [degrees[i] for i in self.sorted_nodes]
        self.n = len(degrees)
        self.k = k

    def dynamic_programming(self):
        n = self.n
        k = self.k
        dp = [float('inf')] * (n + 1)  # dp[i] = minimal cost for the first i nodes
        dp[0] = 0
        I = self._precompute_I()
        best_t = [-1] * (n + 1)  # Track the best split points for reconstruction

        # Dynamic programming to compute minimal anonymization cost
        for i in range(1, n + 1):
            if i < k:
                dp[i] = I[0][i-1]
                best_t[i] = 0  # Group [0, i-1]
            else:
                start = max(k, i - 2*k + 1)
                end = i - k + 1
                if start < end:
                    min_cost = float('inf')
                    best_split = start
                    for t in range(start, end):
                        current_cost = dp[t] + I[t][i-1]
                        if current_cost < min_cost:
                            min_cost = current_cost
                            best_split = t
                    dp[i] = min(dp[i], min_cost)
                    best_t[i] = best_split
                else:
                    dp[i] = I[0][i-1]
                    best_t[i] = 0  # Fallback to grouping all nodes [0, i-1]

        # Reconstruct the anonymized sequence using best_t
        anonymized = [0] * n  # Preallocate anonymized sequence
        current = n
        while current > 0:
            t = best_t[current]
            group_start = t
            group_end = current - 1  # Because current corresponds to i in dp[i]
            assigned_degree = max(self.sorted_degrees[group_start:group_end+1])  # Maximum degree in the group
            
            # Map anonymized degrees back to original node indices
            for idx in range(group_start, group_end + 1):
                original_node = self.sorted_nodes[idx]
                anonymized[original_node] = assigned_degree
            current = t

        return anonymized

    def greedy(self):
        groups = []
        i = 0
        while i < self.n:
            current_degree = self.sorted_degrees[i]
            j = i

            # Expand the group until it has at least k elements
            while (j + 1 < self.n and (j - i + 1) < self.k):
                j += 1

            # Enforce 2k-1 group size restriction explicitly
            j = min(j, self.n - 1)
            if (j - i + 1) > 2 * self.k - 1:
                j = i + 2 * self.k - 2

            # Set all degrees in this group to the maximum degree in the group
            max_deg = self.sorted_degrees[i]
            groups.append((i, j, max_deg))
            i = j + 1

        # Construct the anonymized list clearly and safely
        anonymized = [0] * self.n  # Preallocate the anonymized list
        for start, end, deg in groups:
            for idx in range(start, end + 1):
                original_node = self.sorted_nodes[idx]
                anonymized[original_node] = deg

        return anonymized

    def _precompute_I(self):
        I = np.full((self.n, self.n), np.inf)
        for i in range(self.n):
            for j in range(i, self.n):
                group_size = j - i + 1
                if self.k <= group_size <= 2 * self.k - 1:  # Enforce group size restriction
                    max_degree = self.sorted_degrees[i]
                    cost = sum(max_degree - self.sorted_degrees[x] for x in range(i, j+1))
                    I[i][j] = cost
        return I

In [107]:
# Generate a random original graph (Barabasi-Albert model)
k = 3  # Ensure at least k-anonymity
num_nodes = 20
# edges_per_new_node = 5
# original_graph = nx.barabasi_albert_graph(num_nodes, edges_per_new_node)

p = 0.3  # Adjust probability to control density
original_graph = nx.erdos_renyi_graph(num_nodes, p)


# Extract the degree sequence of the original graph
original_degrees = sorted([d for _, d in original_graph.degree()], reverse=True)

# Initialize the DegreeAnonymizer
degree_anonymizer = DegreeAnonymizer(original_degrees, k)

# Perform degree anonymization using Dynamic Programming
anonymized_degrees_dp = degree_anonymizer.dynamic_programming()

# Perform degree anonymization using Greedy
anonymized_degrees_greedy = degree_anonymizer.greedy()

# Create DataFrames to display results
df_large_dp = pd.DataFrame({
    "Original Degrees": original_degrees,
    "Anonymized Degrees (DP)": anonymized_degrees_dp
})

df_large_greedy = pd.DataFrame({
    "Original Degrees": original_degrees,
    "Anonymized Degrees (Greedy)": anonymized_degrees_greedy
})

# Display results
print("Dynamic Programming Results:")
print(df_large_dp)

print("\nGreedy Results:")
print(df_large_greedy)

Dynamic Programming Results:
    Original Degrees  Anonymized Degrees (DP)
0                 10                       10
1                  9                       10
2                  9                       10
3                  9                       10
4                  8                        8
5                  8                        8
6                  8                        8
7                  7                        8
8                  6                        6
9                  6                        6
10                 6                        6
11                 6                        6
12                 6                        6
13                 5                        5
14                 5                        5
15                 4                        5
16                 4                        4
17                 4                        4
18                 3                        4
19                 3                        4

Gree

### 2. Graph Construction (Supergraph Algorithm)

In [96]:
class GraphConstructor:
    def __init__(self, G, anonymized_degrees):
        # Relabel nodes explicitly to integers (0...n-1)
        self.G = nx.convert_node_labels_to_integers(G, ordering='sorted')
        self.node_list = sorted(self.G.nodes())
        self.n = len(self.node_list)

        # Ensure anonymized_degrees match node count
        if len(anonymized_degrees) != self.n:
            raise ValueError(f"Length mismatch: anonymized_degrees ({len(anonymized_degrees)}) and graph nodes ({self.n}) must match.")
        
        self.anonymized_degrees = anonymized_degrees

    def supergraph(self):
        print(Fore.CYAN + "[DEBUG] Supergraph construction started.")
        # Calculate residual degrees explicitly
        residual = [
            self.anonymized_degrees[i] - self.G.degree(self.node_list[i])
            for i in range(self.n)
        ]
        print(Fore.YELLOW + f"[DEBUG] Initial residual degrees: {residual}")

        # Validate residual degrees
        if any(r < 0 for r in residual):
            print(Fore.RED + "[ERROR] Negative residual degrees detected at start.")
            print(Fore.RED + f"[DEBUG] Residual degrees: {residual}")
            return None

        edges = set(self.G.edges())
        iteration = 0
        while sum(residual) > 0:
            iteration += 1
            print(Fore.CYAN + f"[DEBUG] Supergraph iteration {iteration}, total residual sum: {sum(residual)}")
            progress = False

            # First pass: existing neighbors
            for idx, v in enumerate(self.node_list):
                if residual[idx] <= 0:
                    continue

                candidates = [
                    u for u in self.G.neighbors(v)
                    if residual[self.node_list.index(u)] > 0 and (v, u) not in edges and (u, v) not in edges
                ]
                candidates.sort(key=lambda x: -residual[self.node_list.index(x)])

                for u in candidates:
                    u_idx = self.node_list.index(u)
                    if residual[idx] <= 0 or residual[u_idx] <= 0:
                        continue
                    edges.add((v, u))
                    residual[idx] -= 1
                    residual[u_idx] -= 1
                    progress = True
                    print(Fore.GREEN + f"[DEBUG] (Existing neighbors) Added edge ({v}, {u}). Residual now: node {v}: {residual[idx]}, node {u}: {residual[u_idx]}")

            # Connect non-neighbors if needed
            for idx, v in enumerate(self.node_list):
                while residual[idx] > 0:
                    candidates = [
                        u for u in self.node_list
                        if u != v and residual[self.node_list.index(u)] > 0
                        and not self.G.has_edge(v, u)
                        and (v, u) not in edges and (u, v) not in edges
                    ]
                    if not candidates:
                        print(Fore.RED + "[ERROR] No further edge additions possible.")
                        print(Fore.RED + f"[DEBUG] Residual degrees at failure: {residual}")
                        return None
                    candidates.sort(key=lambda x: -residual[self.node_list.index(x)])
                    u = candidates[0]
                    u_idx = self.node_list.index(u)
                    edges.add((v, u))
                    residual[idx] -= 1
                    residual[u_idx] -= 1
                    progress = True
                    print(Fore.YELLOW + f"[DEBUG] (Non-neighbors) Added edge ({v}, {u}). Residual now: node {v}: {residual[idx]}, node {u}: {residual[u_idx]}")

            iteration += 1
            if not progress:
                print(Fore.RED + "[ERROR] Supergraph construction stuck: no progress made.")
                print(Fore.RED + f"[DEBUG] Final residual degrees: {residual}")
                return None

        # Successfully constructed the supergraph
        G_super = nx.Graph()
        G_super.add_nodes_from(self.node_list)
        G_super.add_edges_from(edges)

        constructed_degrees = [deg for _, deg in G_super.degree()]
        print(Fore.GREEN + "[DEBUG] Successfully constructed supergraph.")
        print(Fore.GREEN + f"[DEBUG] Constructed degrees: {sorted(constructed_degrees, reverse=True)}")
        print(Fore.GREEN + f"[DEBUG] Expected anonymized degrees: {sorted(self.anonymized_degrees, reverse=True)}")

        return G_super


### 3. Probing Scheme

In [97]:
def probing_scheme(G, k, anonymizer_func, max_iter=100):
    G = nx.convert_node_labels_to_integers(G)
    original_degrees = [d for _, d in G.degree()]
    n = len(original_degrees)
    
    print(Fore.CYAN + "\n--- Step 0: Initialization ---")
    print(Fore.YELLOW + f"[DEBUG] Original graph has {n} nodes.")
    print(Fore.YELLOW + f"[DEBUG] Original degrees (node: degree): { {i: original_degrees[i] for i in range(n)} }")

    modified_nodes = set()
    best_G_anon = None
    best_edge_overlap = 0.0

    current_degrees = original_degrees.copy()  # <-- persistently updated degrees!

    for iteration in range(max_iter):
        print(Fore.CYAN + f"\n--- Iteration {iteration + 1}/{max_iter} ---")
        print(Fore.YELLOW + f"[DEBUG] Current original degrees: {current_degrees}")

        # Step 1: Anonymization
        anonymizer = DegreeAnonymizer(current_degrees, k)
        anonymized_degrees = anonymizer_func(anonymizer)
        print(Fore.GREEN + "[DEBUG] Step 1: Anonymization Result:")
        print(Fore.GREEN + f"[DEBUG] Anonymized degrees: {anonymized_degrees}")

        # Step 2: Graphicality check
        is_graphical_flag = is_graphical(anonymized_degrees)
        print(Fore.MAGENTA + f"[DEBUG] Step 2: Graphicality check result: {is_graphical_flag}")

        if not is_graphical_flag:
            print(Fore.RED + "[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.")
            
            # Adjust the lowest-degree node that hasn't been modified yet
            sorted_nodes = sorted(range(n), key=lambda x: (current_degrees[x], x in modified_nodes))
            node_to_adjust = next((node for node in sorted_nodes if node not in modified_nodes), None)
            
            if node_to_adjust is None:
                print(Fore.RED + "[DEBUG] All nodes modified. Resetting modified_nodes set.")
                modified_nodes.clear()
                node_to_adjust = sorted_nodes[0]
            
            old_degree = current_degrees[node_to_adjust]
            new_degree = min(old_degree + 1, n - 1)
            current_degrees[node_to_adjust] = new_degree
            modified_nodes.add(node_to_adjust)

            print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
            print(Fore.RED + f"[DEBUG] Modified nodes: {modified_nodes}")
            continue

        # Step 4: Graph construction
        print(Fore.GREEN + "[DEBUG] Step 4: Graph construction attempt.")
        constructor = GraphConstructor(G, anonymized_degrees)
        G_anon = constructor.supergraph()

        if G_anon is not None:
            print(Fore.GREEN + "[DEBUG] Graph construction SUCCEEDED.")
            
            # Step 5: Greedy_Swap optimization
            print(Fore.BLUE + "[DEBUG] Step 5: Greedy_Swap optimization started.")
            swapper = GreedySwap(G, G_anon)
            G_anon_optimized = swapper.swap()
            print(Fore.BLUE + "[DEBUG] Greedy_Swap optimization completed.")

            # Edge overlap
            original_edges = set(G.edges())
            anonymized_edges = set(G_anon_optimized.edges())
            edge_overlap = len(original_edges & anonymized_edges) / len(original_edges)
            print(Fore.BLUE + f"[DEBUG] Edge overlap ratio: {edge_overlap:.2f}")

            return G_anon_optimized

        # If construction failed:
        print(Fore.RED + "[DEBUG] Graph construction FAILED. Adjusting degrees again.")
        sorted_nodes = sorted(range(n), key=lambda x: (x in modified_nodes, current_degrees[x]))
        node_to_adjust = sorted_nodes[0]
        old_degree = current_degrees[node_to_adjust]
        new_degree = min(old_degree + 1, n - 1)
        current_degrees[node_to_adjust] = new_degree
        modified_nodes.add(node_to_adjust)

        print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
        print(Fore.RED + f"[DEBUG] Updated original degrees: {current_degrees}")

    print(Fore.RED + "[DEBUG] --- Probing scheme FAILED after maximum iterations ---")
    print(Fore.RED + "[DEBUG] Falling back to the best anonymized graph found.")
    return best_G_anon if best_G_anon is not None else nx.complete_graph(n)


In [98]:
def validate_k_anonymity(d_anon, k):
    groups = {}
    for degree in d_anon:
        groups[degree] = groups.get(degree, 0) + 1
    for count in groups.values():
        if count < k:
            return False
    return True

In [99]:
def evaluate_edge_overlap(G_original, G_anon):
    original_edges = set(G_original.edges())
    anonymized_edges = set(G_anon.edges())
    edge_intersect = len(original_edges & anonymized_edges) / len(original_edges)
    return edge_intersect

In [100]:
def is_graphical(degrees):
    """
    Check if a degree sequence is graphical using the Erdős–Gallai theorem.

    Parameters:
        degrees (list): A list of integers representing the degree sequence.

    Returns:
        bool: True if the sequence is graphical, False otherwise.
    """
    # Step 1: Sort the degree sequence in non-increasing order
    degrees = sorted(degrees, reverse=True)
    n = len(degrees)

    # Step 2: Check if the sum of degrees is even
    total_degrees = sum(degrees)
    if total_degrees % 2 != 0:
        return False  # The sum of degrees must be even for a graph to exist

    # Step 3: Apply the Erdős–Gallai condition
    for k in range(1, n + 1):
        sum_deg = sum(degrees[:k])  # Sum of the first k degrees
        max_sum = k * (k - 1) + sum(min(d, k) for d in degrees[k:])  # Maximum possible sum
        if sum_deg > max_sum:
            return False  # Erdős–Gallai condition violated

    return True  # The sequence is graphical

In [106]:
# Step 1: Define the anonymizer functions
def dp_anonymizer(anonymizer):
    return anonymizer.dynamic_programming()

def greedy_anonymizer(anonymizer):
    return anonymizer.greedy()

# Step 2: Call the Probing scheme with the desired anonymizer function
G_anon_dp = probing_scheme(original_graph, k, anonymizer_func=dp_anonymizer)  # Using Dynamic Programming
G_anon_greedy = probing_scheme(original_graph, k, anonymizer_func=greedy_anonymizer)  # Using Greedy

# Step 3: Display Results
if G_anon_dp is not None:
    print("Anonymization Successful with Dynamic Programming!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_dp.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_dp.edges())
else:
    print("Anonymization failed with Dynamic Programming after max iterations.")

if G_anon_greedy is not None:
    print("Anonymization Successful with Greedy!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_greedy.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_greedy.edges())
else:
    print("Anonymization failed with Greedy after max iterations.")


--- Step 0: Initialization ---
[DEBUG] Original graph has 20 nodes.
[DEBUG] Original degrees (node: degree): {0: 4, 1: 5, 2: 4, 3: 6, 4: 6, 5: 4, 6: 2, 7: 3, 8: 0, 9: 3, 10: 3, 11: 1, 12: 5, 13: 5, 14: 4, 15: 6, 16: 6, 17: 3, 18: 4, 19: 4}

--- Iteration 1/100 ---
[DEBUG] Current original degrees: [4, 5, 4, 6, 6, 4, 2, 3, 0, 3, 3, 1, 5, 5, 4, 6, 6, 3, 4, 4]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees: [4, 6, 4, 6, 6, 4, 3, 3, 3, 3, 3, 3, 6, 6, 4, 6, 6, 3, 4, 4]
[DEBUG] Step 2: Graphicality check result: False
[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.
[DEBUG] Adjusted node 8: degree 0 → 1
[DEBUG] Modified nodes: {8}

--- Iteration 2/100 ---
[DEBUG] Current original degrees: [4, 5, 4, 6, 6, 4, 2, 3, 1, 3, 3, 1, 5, 5, 4, 6, 6, 3, 4, 4]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees: [4, 6, 4, 6, 6, 4, 3, 3, 3, 3, 3, 3, 6, 6, 4, 6, 6, 3, 4, 4]
[DEBUG] Step 2: Graphicality check result: False
[DEBUG] Step 3: Graphicality FAILED, adjustin

### 3. Greedy_Swap Algorithm

In [108]:
class GreedySwap:
    def __init__(self, G_original, G_anon):
        self.G_original = G_original
        self.G_anon = G_anon
        self.original_edges = set(G_original.edges())
        self.common_edges = set(G_original.edges()) & set(G_anon.edges())
        print(Fore.YELLOW + f"[DEBUG] Initial common edges count: {len(self.common_edges)}")

    def find_max_swap(self):
        max_gain = -float('inf')  # Allow swaps even if gain is 0
        best_swap = None
        edges = list(self.G_anon.edges())
        sampled_edges = random.sample(edges, min(len(edges), len(edges)))

        for i in range(len(sampled_edges)):
            for j in range(i + 1, len(sampled_edges)):
                u1, v1 = sampled_edges[i]
                u2, v2 = sampled_edges[j]

                if len({u1, v1, u2, v2}) < 4:
                    continue

                swap_options = [
                    ((u1, u2), (v1, v2)),
                    ((u1, v2), (v1, u2))
                ]

                for new_e1, new_e2 in swap_options:
                    if self.G_anon.has_edge(*new_e1) or self.G_anon.has_edge(*new_e2):
                        continue

                    gain = 0
                    gain += (new_e1 in self.original_edges) + (new_e2 in self.original_edges)
                    gain -= ((u1, v1) in self.common_edges) + ((u2, v2) in self.common_edges)

                    # ✅ Allow swaps even if gain is 0 to create future opportunities
                    if gain >= max_gain:
                        max_gain = gain
                        best_swap = ((u1, v1), (u2, v2), new_e1, new_e2)

        print(Fore.CYAN + f"[DEBUG] find_max_swap completed. Best gain: {max_gain}, Best swap: {best_swap}")
        return max_gain, best_swap  # Allow swaps with gain >= 0

    def swap(self, max_iter=100):
        for iteration in range(max_iter):
            print(Fore.CYAN + f"[DEBUG] Swap iteration {iteration + 1}")
            gain, swap_edges = self.find_max_swap()

            # ✅ If no beneficial swaps, force a random swap every few iterations
            if swap_edges is None or gain < 0:
                if iteration % 5 == 0:  # Force a swap every 5 iterations
                    print(Fore.RED + "[DEBUG] No beneficial swap found. Forcing a random swap.")
                    edges = list(self.G_anon.edges())
                    if len(edges) > 1:
                        e1, e2 = random.sample(edges, 2)
                        u1, v1 = e1
                        u2, v2 = e2
                        new_e1, new_e2 = (u1, u2), (v1, v2)
                        if not self.G_anon.has_edge(*new_e1) and not self.G_anon.has_edge(*new_e2):
                            swap_edges = (e1, e2, new_e1, new_e2)
                else:
                    print(Fore.RED + "[DEBUG] No beneficial swap found. Ending optimization.")
                    break

            e1, e2, new_e1, new_e2 = swap_edges
            print(Fore.BLUE + f"[DEBUG] Performing swap: remove {e1}, {e2}; add {new_e1}, {new_e2}")

            self.G_anon.remove_edges_from([e1, e2])
            self.G_anon.add_edges_from([new_e1, new_e2])

            # Update common edges
            self.common_edges.discard(e1)
            self.common_edges.discard(e2)
            if new_e1 in self.original_edges:
                self.common_edges.add(new_e1)
            if new_e2 in self.original_edges:
                self.common_edges.add(new_e2)

            print(Fore.YELLOW + f"[DEBUG] Common edges count after swap: {len(self.common_edges)}")

        return self.G_anon


### 5. Evaluation Metrics

In [109]:
def evaluate(G_original, G_anon):
    metrics = {}
    
    # L1 norm of degree differences
    l1 = sum(abs(G_original.degree(n) - G_anon.degree(n)) for n in G_original.nodes())
    metrics['L1'] = l1
    
    # Clustering Coefficient
    cc_orig = nx.average_clustering(G_original)
    cc_anon = nx.average_clustering(G_anon)
    metrics['CC_original'] = cc_orig
    metrics['CC_anon'] = cc_anon
    
    # Average Path Length (largest connected component)
    def largest_connected_component_apl(G):
        largest_cc = max(nx.connected_components(G), key=len)
        return nx.average_shortest_path_length(G.subgraph(largest_cc))
    
    apl_orig = largest_connected_component_apl(G_original)
    apl_anon = largest_connected_component_apl(G_anon)
    metrics['APL_original'] = apl_orig
    metrics['APL_anon'] = apl_anon
    
    # Edge Intersection
    if len(G_original.edges()) == 0:
        edge_intersect = 0
    else:
        edge_intersect = len(set(G_original.edges()) & set(G_anon.edges())) / len(G_original.edges())
    metrics['Edge_Intersection'] = edge_intersect
    
    return metrics

### 6. Usage

In [112]:
# Generate a random original graph (Barabasi-Albert model)
k = 5  # Ensure at least k-anonymity
num_nodes = 30

# k = 3  # Ensure at least k-anonymity
# num_nodes = 200

p = 0.2  # Adjust probability to control density
original_graph = nx.erdos_renyi_graph(num_nodes, p)

# Step 1: Define the anonymizer functions
def dp_anonymizer(anonymizer):
    return anonymizer.dynamic_programming()

def greedy_anonymizer(anonymizer):
    return anonymizer.greedy()

# Step 2: Call the Probing scheme with the desired anonymizer function
G_anon_dp = probing_scheme(original_graph, k, anonymizer_func=dp_anonymizer)  # Using Dynamic Programming
G_anon_greedy = probing_scheme(original_graph, k, anonymizer_func=greedy_anonymizer)  # Using Greedy

# Step 3: Display Results
if G_anon_dp is not None:
    print("Anonymization Successful with Dynamic Programming!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_dp.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_dp.edges())
else:
    print("Anonymization failed with Dynamic Programming after max iterations.")

if G_anon_greedy is not None:
    print("Anonymization Successful with Greedy!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_greedy.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_greedy.edges())
else:
    print("Anonymization failed with Greedy after max iterations.")

# Evaluate results for Greedy
if G_anon_greedy is not None:
    metrics_greedy = evaluate(original_graph, G_anon_greedy)
    print("\nEvaluation Metrics (Greedy):", metrics_greedy)
else:
    print("\nGreedy Anonymization failed.")

# Evaluate results for Dynamic Programming
if G_anon_dp is not None:
    metrics_dp = evaluate(original_graph, G_anon_dp)
    print("\nEvaluation Metrics (Dynamic Programming):", metrics_dp)
else:
    print("\nDynamic Programming Anonymization failed.")


--- Step 0: Initialization ---
[DEBUG] Original graph has 30 nodes.
[DEBUG] Original degrees (node: degree): {0: 6, 1: 4, 2: 9, 3: 8, 4: 10, 5: 3, 6: 6, 7: 6, 8: 6, 9: 4, 10: 6, 11: 2, 12: 10, 13: 1, 14: 2, 15: 4, 16: 5, 17: 6, 18: 4, 19: 7, 20: 7, 21: 3, 22: 5, 23: 9, 24: 5, 25: 6, 26: 7, 27: 8, 28: 4, 29: 5}

--- Iteration 1/100 ---
[DEBUG] Current original degrees: [6, 4, 9, 8, 10, 3, 6, 6, 6, 4, 6, 2, 10, 1, 2, 4, 5, 6, 4, 7, 7, 3, 5, 9, 5, 6, 7, 8, 4, 5]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees: [7, 4, 10, 10, 10, 3, 7, 6, 6, 4, 6, 3, 10, 3, 3, 4, 6, 6, 4, 7, 7, 3, 6, 10, 6, 6, 7, 10, 4, 6]
[DEBUG] Step 2: Graphicality check result: True
[DEBUG] Step 4: Graph construction attempt.
[DEBUG] Supergraph construction started.
[DEBUG] Initial residual degrees: [1, 0, 1, 2, 0, 0, 1, 0, 0, 0, 0, 1, 0, 2, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 2, 0, 1]
[DEBUG] Supergraph iteration 1, total residual sum: 16
[DEBUG] (Non-neighbors) Added edge (0, 3). Residual now: no